# Importar os módulos necessários

In [52]:
import warnings
warnings.filterwarnings("ignore")

In [53]:
import pandas as pd
#necessario por causa do metodo de avaliaçao
from sklearn.model_selection import KFold
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier # decision trees for classification
from sklearn.neural_network import  MLPClassifier # neural networks for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVC # support vector machines for classification

# Definir as Métricas para Avaliação dos Modelos

In [54]:
#metricas regressao
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def custom_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return mse, mae, r2


In [55]:
#metricas classificação
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [56]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import make_scorer
import numpy as np

# Função para calcular o RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Dicionário de métricas de regressão
REGRESSION_METRICS = {
    "MSE": make_scorer(mean_squared_error),
    "MAE": make_scorer(mean_absolute_error),
    "R2": make_scorer(r2_score),
    "RMSE": make_scorer(root_mean_squared_error)  # Adicionando o RMSE
}


In [57]:
METRICS = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score),
        "recall": make_scorer(recall_score),
        "f1": make_scorer(f1_score),
        "AUC": make_scorer(roc_auc_score, needs_proba=True),
        "specificity": make_scorer(specificity_score),
        "kappa":make_scorer(cohen_kappa_score)
}

# Ler o Conjunto de Dados

In [58]:
d = pd.read_csv("Acenario2Ideia2Marcas.csv")
d

,id_transmissao,id_body_type,id_modelos,id_marca,mileage,ano,id_city,id_insurance,seats,engine_capacity,id_fuel_type,kms_driven,max_power,id_owner_type,resale_price_Lakh
0,0.130634,0.363138,3.089198,1.232415,23.84000,2019.0,1.095161,0.256167,5.0,1199.0,0.180811,30910.0,83.8,0.151316,5.66
1,0.130634,0.600263,2.406315,0.978303,17.50000,2018.0,1.280558,1.309078,5.0,1199.0,0.180811,48089.0,88.7,1.355345,6.64
2,0.130634,0.577680,2.586523,0.978303,17.40000,2015.0,0.923765,0.256167,5.0,1497.0,0.180811,51000.0,117.3,0.619644,5.65
3,0.585160,0.600263,3.653470,1.232415,14.08000,2021.0,0.923765,0.256167,7.0,1956.0,0.514536,30000.0,167.6,0.151316,23.00
4,0.585160,0.363138,2.750380,0.542880,21.40000,2019.0,1.170596,1.309078,5.0,1197.0,0.180811,61113.0,83.1,0.151316,6.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13503,0.585160,1.350274,3.653470,1.748574,19.49405,2021.0,1.027129,0.256167,7.0,2199.0,0.514536,80000.0,197.2,0.619644,26.50
13504,0.130634,0.363138,2.954500,0.676204,19.49405,2017.0,1.222106,0.256167,5.0,1197.0,0.180811,17923.0,81.8,0.151316,5.87
13505,0.130634,0.577680,3.285493,1.694428,19.49405,2017.0,1.027129,0.446914,5.0,1498.0,0.514536,63389.0,108.6,0.619644,7.43
13506,0.130634,0.600263,2.750380,0.542880,19.49405,2017.0,0.868615,0.446914,5.0,1248.0,0.514536,40000.0,88.5,0.619644,9.45


In [59]:
X, y = d.drop("resale_price_Lakh", axis=1), d["resale_price_Lakh"]

# Definir o Método de Validação Cruzada

In [60]:
#Classificação
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)


# Implementação dos Algoritmos de Machine Learning

### Árvores de Decisão

In [61]:
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Definir o regressor e o método de validação cruzada
dt_regressor = DecisionTreeRegressor(max_depth=10, random_state=1234)
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = {k: -v.mean() for k, v in scores.items()}  # Negar os valores para tornar os scores positivos
pd.DataFrame(dt_scores, index=[0])



,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,-0.070463,-0.00331,-10.584921,-1.353971,-0.881943,-3.185971


In [62]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Continue com a validação cruzada e a avaliação de métricas
dt_regressor = DecisionTreeRegressor(max_depth=7, random_state=1234)
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T





,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.042315,0.000959,12.283029,1.58793,0.864866,3.427548


In [63]:


# Criar um modelo de regressão de árvore de decisão
dt_regressor = DecisionTreeRegressor(max_depth=3, random_state=1234)

# Estratégia de validação cruzada (KFold) para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Continuar com a validação cruzada e avaliação de métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T



,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_AUC,test_specificity,test_kappa
0,0.016952,0.008374,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Redes Neuronais

In [64]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

# Criar uma Rede Neural para regressão
nn_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=20, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_nn = cross_validate(nn_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,1.689642,0.004417,65.414124,5.006062,0.262714,7.807414


In [65]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

# Criar uma Rede Neural para regressão
nn_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=20, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_nn = cross_validate(nn_regressor, X, y, cv=kf, scoring=METRICS)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_AUC,test_specificity,test_kappa
0,1.586674,0.006828,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Naive Bayes

In [66]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error



# Criar um regressor RandomForest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T


KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error



# Criar um regressor RandomForest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=kf, scoring=METRICS)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_AUC,test_specificity,test_kappa
0,14.136671,0.044294,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Support Vector Machine